In [1]:
import math
from itertools import combinations
from music21 import *

In [4]:
def roughness(notes):
    '''
    Calculate the Roughness of notes according to (Vassilakis, 2001, 2005)
    Reference: http://www.acousticslab.org/learnmoresra/moremodel.html
    '''
    def roughness_calculation(f1,f2,a1,a2):
        fmin = min(f1,f2)
        fmax = max(f1,f2)
        amin = min(a1,a2)
        amax = max(a1,a2)
        X = amin*amax
        Y = 2*amin/(amin+amax)
        b1 = 3.5
        b2 = 5.75
        s1 = 0.0207
        s2=18.96
        s = 0.24/(s1*fmin+s2)
        Z = math.exp(-b1*s*(fmax-fmin)) - math.exp(-b2*s*(fmax-fmin))
        return 100*(X**0.1)*0.5*(Y**3.11)*Z
    def keys2freq(keys):
    # 1 by 1 key to number
        key_mapping = {"C": 0, "D": 2, "E": 4, "F": 5, "G": 7, "A": 9, "B": 11}
        def halfsteps(key):
            ans = 0
            try:
                octave = int(key[-1])
            except:
                print("You must provide the octave.")
                return 0
            key = key.upper()[:-1]
            num = key_mapping[key[0]]
            modifier = len(key)
            if modifier == 1:
                ans = (octave-1)*12 +num
            elif key[1] == "#":
                ans =  (octave-1)*12 + (num + (modifier - 1)) % 12
            elif key[1] == "B" or key[1] == "-":
                ans = (octave-1)*12 +(num - (modifier - 1)) % 12
            elif key[1] == "X":
                ans = (octave-1)*12 +(num + (modifier - 1) * 2) % 12
            return ans - 45
        a = 2 ** (1/12)
        f0 = 440 #A4
        return [round(f0*(a**halfsteps(key)),3) for key in keys]
    frequencies = keys2freq(notes)
    frequencies.sort()
    ans = 0
    count = 0
    for combo in combinations(frequencies,2):
        count += 1
        ans += roughness_calculation(combo[0],combo[1],1,1)
#     for i in range(1,len(frequencies)):
#         ans += roughness_calculation(frequencies[1],frequencies[0],1,1)
    return ans

In [9]:
def roughness2(notes):
    '''
    Calculate the Roughness of notes according to sum of ideal ratio N+M
    Reference: https://www.researchgate.net/publication/276905584_Measuring_Musical_Consonance_and_Dissonance
    '''
    def interval_to_ratio(interval):
        interval_ratio_mapping = {
            0:1+1,
            1:18+17,
            2:9+8,
            3:6+5,
            4:5+4,
            5:4+3,
            6:17+12,
            7:3+2,
            8:8+5,
            9:5+3,
            10:16+9,
            11:17+9,
        }
        interval_pitch_mapping = {
            1:0,
            2:2,
            3:4,
            4:5,
            5:7,
            6:9,
            7:11,
            8:12
        }
        ans = interval_pitch_mapping[int(interval[-1])]
        if int(interval[-1]) in [4,5,8]:
            intname = interval[:-1]
            if intname == "dd":
                ans -= 2
            elif intname == "d":
                ans -= 1
            elif intname == "A":
                ans += 1
            elif intname == "AA":
                ans += 2
        else:
            intname = interval[:-1]
            if intname == "m":
                ans -= 1
            elif intname == "d":
                ans -= 2
            elif intname == "A":
                ans += 1
            elif intname == "AA":
                ans += 2
        ans = ans%12
        return interval_ratio_mapping[ans]
    ans = 0
    for combo in combinations(notes,2):
        n1 = note.Note(combo[0])
        n2 = note.Note(combo[1])
        xinterval = interval.Interval(noteStart=n1,noteEnd=n2)
        ans += interval_to_ratio(xinterval.semiSimpleName)
    return ans/len(notes) if len(notes)!= 0 else 0
    

In [6]:
roughness2([])

0

In [12]:
notes=["E-4","C3","G2","D2","C3","A-4","F4"]
substring = [notes[i: j] for i in range(len(notes)) for j in range(i + 1, len(notes) + 1)]
analysis=[]
for idx,e in enumerate(substring):
    e = list(dict.fromkeys(e))
    y = substring[idx-1]
    y = list(dict.fromkeys(y))
    if idx==0:
        analysis.append([roughness2(e)-0,roughness2(e),e])
    else:
        analysis.append([roughness2(e)-roughness2(y),roughness2(e),e])
import pandas as pd
pd.DataFrame(analysis)

,0,1,2
0,0.000000,0.000000,[E-4]
1,5.500000,5.500000,"[E-4, C3]"
2,4.833333,10.333333,"[E-4, C3, G2]"
3,14.166667,24.500000,"[E-4, C3, G2, D2]"
4,0.000000,24.500000,"[E-4, C3, G2, D2]"
5,11.900000,36.400000,"[E-4, C3, G2, D2, A-4]"
6,5.766667,42.166667,"[E-4, C3, G2, D2, A-4, F4]"
7,-42.166667,0.000000,[C3]
8,3.500000,3.500000,"[C3, G2]"
9,9.500000,13.000000,"[C3, G2, D2]"


In [8]:
notes=["E-4","C3","G2","D2","C2","A-4","F4"]
substring = [notes[i: j] for i in range(len(notes)) for j in range(i + 1, len(notes) + 1)]
analysis=[]
for idx,e in enumerate(substring):
    if idx==0:
        analysis.append([roughness(e)-0,roughness(e),e])
    else:
        analysis.append([roughness(e)-roughness(substring[idx-1]),roughness(e),e])
import pandas as pd
pd.DataFrame(analysis)

,0,1,2
0,0.000000,0.000000,[E-4]
1,0.045526,0.045526,"[E-4, C3]"
2,7.676174,7.721699,"[E-4, C3, G2]"
3,12.407532,20.129231,"[E-4, C3, G2, D2]"
4,17.173300,37.302531,"[E-4, C3, G2, D2, C2]"
5,1.421802,38.724333,"[E-4, C3, G2, D2, C2, A-4]"
6,12.354707,51.079040,"[E-4, C3, G2, D2, C2, A-4, F4]"
7,-51.079040,0.000000,[C3]
8,7.666340,7.666340,"[C3, G2]"
9,12.404622,20.070962,"[C3, G2, D2]"


In [4]:
roughness(["C3","E3","G3"]),roughness(["C3","E3","G3","C4"]),roughness(["C3","C4","C5"]),roughness(["C4","C#4","D4"])

(19.05554456831978, 24.12970487852886, 0.30775484111978496, 25.588765105965702)

In [5]:
roughness2(["C3","E3","G3"]),roughness2(["C3","E3","G3","C4"]),roughness2(["C3","C4","C5"]),roughness2(["C4","C#4","D4"])

(8.333333333333334, 12.0, 3.0, 29.0)

In [6]:
roughness(["C2","E2","G2"]),roughness(["C3","E3","G3"])

(25.4075578627804, 19.05554456831978)

In [7]:
roughness2(["C2","E2","G2"]),roughness2(["C3","E3","G3"])

(8.333333333333334, 8.333333333333334)

In [8]:
roughness(["C3","F3","A3"]),roughness(["G3","D4","F4","G4"]),roughness(["A3","C4","F#4"])

(13.60247543171581, 15.9631712052252, 8.271100535423262)

In [9]:
roughness2(["C3","F3","A3"]),roughness2(["G3","D4","F4","G4"]),roughness2(["A3","C4","F#4"])

(8.0, 17.0, 16.0)

In [10]:
roughness(["C3","E3"]),roughness(["C3","Eb3"])

(7.6471649145129605, 8.817537907843318)

In [11]:
roughness2(["C3","E3"]),roughness2(["C3","Eb3"])

(4.5, 5.5)

In [ ]:
def change_in_roughness(self):
        def roughness(notes):
            '''
            Calculate the Roughness of notes according to sum of ideal ratio N+M
            Reference: https://www.researchgate.net/publication/276905584_Measuring_Musical_Consonance_and_Dissonance
            '''
            def interval_to_ratio(interval):
                interval_ratio_mapping = {
                    0:1+1,
                    1:18+17,
                    2:9+8,
                    3:6+5,
                    4:5+4,
                    5:4+3,
                    6:17+12,
                    7:3+2,
                    8:8+5,
                    9:5+3,
                    10:16+9,
                    11:17+9,
                }
                interval_pitch_mapping = {
                    1:0,
                    2:2,
                    3:4,
                    4:5,
                    5:7,
                    6:9,
                    7:11,
                    8:12
                }
                ans = interval_pitch_mapping[int(interval[-1])]
                if int(interval[-1]) in [4,5,8]:
                    intname = interval[:-1]
                    if intname == "dd":
                        ans -= 2
                    elif intname == "d":
                        ans -= 1
                    elif intname == "A":
                        ans += 1
                    elif intname == "AA":
                        ans += 2
                else:
                    intname = interval[:-1]
                    if intname == "m":
                        ans -= 1
                    elif intname == "d":
                        ans -= 2
                    elif intname == "A":
                        ans += 1
                    elif intname == "AA":
                        ans += 2
                ans = ans%12
                return interval_ratio_mapping[ans]
            ans = 0
            for combo in combinations(notes,2):
                n1 = note.Note(combo[0])
                n2 = note.Note(combo[1])
                xinterval = interval.Interval(noteStart=n1,noteEnd=n2)
                ans += interval_to_ratio(xinterval.semiSimpleName)
            return ans/len(notes) if len(notes)!= 0 else 0
        notelist1 = []
        for i in range(self.notelistfirst,self.latestbeatfirst):
            notelist1.append(self.notes[self.current_piece][i]+str(self.octave[self.current_piece][i]))
        notelist2 = notelist1.copy()
        for i in range(self.latestbeatfirst,self.latestbeatlast):
            notelist2.append(self.notes[self.current_piece][i]+str(self.octave[self.current_piece][i]))
        notelist1 = list(dict.fromkeys(notelist1))
        notelist2 = list(dict.fromkeys(notelist2))
        return abs(roughness(notelist2)-roughness(notelist1))
    